In [55]:
import tensorflow as tf
import numpy as np


In [2]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import OneHotEncoder
digits = load_digits()
X = digits.data.astype(np.float32).reshape(-1,8,8,1)
y = digits.target.astype(np.float32).reshape(-1,1)
y = OneHotEncoder().fit_transform(y).todense()
print(X.shape)
print(y.shape)

(1797, 8, 8, 1)
(1797, 10)


In [3]:
def batchGenerator(X,y,batch_size=64):
    i = 0
    while True:
        yield X[i*batch_size:(i+1)*batch_size],y[i*batch_size:(i+1)*batch_size]
        i += 1
        if i>= X.shape[0]//batch_size:
            i=0

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
tf.reset_default_graph()
# Input laye
x_ = tf.placeholder(tf.float32,[None,8,8,1])
y_ = tf.placeholder(tf.float32,[None,10])

In [8]:
#size of filter(height,width), input channel, output channel
conv_w1 = weight_variable([3, 3, 1, 32])
conv_b1 = bias_variable([32])
conv1 = tf.nn.relu(conv2d(x_, conv_w1) + conv_b1)
conv_w2 = weight_variable([3, 3, 32, 64])
conv_b2 = bias_variable([64])
conv2 = tf.nn.relu(conv2d(conv1, conv_w2) + conv_b2)

In [9]:
max1 = max_pool_2x2(conv2)

In [10]:
drop1 = tf.nn.dropout(max1, 0.25)

In [11]:
batch_mean, batch_var = tf.nn.moments(drop1, [0, 1, 2], keep_dims=True)
bn1 = tf.nn.batch_normalization(drop1,batch_mean,
                                   batch_var,
                                   tf.Variable(tf.zeros([64])),
                                   tf.Variable(tf.ones([64])),1e-3)

In [12]:
flat = tf.reshape(bn1, [-1, int(bn1.shape[1]*bn1.shape[2]*bn1.shape[3])])

In [13]:
w1 = weight_variable([int(flat.shape[1]), 256])
b1 = bias_variable([256])

h1 =tf.nn.relu(tf.matmul(flat, w1) + b1)

In [14]:
drop2 = tf.nn.dropout(h1, 0.4)

In [16]:
batch_mean, batch_var = tf.nn.moments(drop2, [0], keep_dims=True)
bn2 = tf.nn.batch_normalization(drop2,batch_mean,
                                   batch_var,
                                   tf.Variable(tf.zeros([256])),
                                   tf.Variable(tf.ones([256])),1e-3)

In [17]:
out_w1 = tf.Variable(tf.random_normal([int(bn2.shape[1]),y.shape[1]]))
out_b1 = tf.Variable(tf.random_normal([y.shape[1]]))
pred = tf.nn.softmax(tf.matmul(bn2,out_w1)+out_b1)

In [21]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=pred))

In [22]:
train_step = tf.train.AdamOptimizer().minimize(loss)

In [23]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_,1),tf.argmax(pred,1)),tf.float32))

In [58]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [59]:
epoch = 100
batch = 64
batch_generator = batchGenerator(X,y,batch)
for i in range(epoch*(X.shape[0]//batch+1)): # 迭代1000个周期
    batch_x,batch_y = batch_generator.__next__()
    sess.run(train_step,feed_dict={x_:batch_x,y_:batch_y})
    if(i%(X.shape[0]//batch+1)==0):
        batch_x,batch_y = batch_generator.__next__()
        res = sess.run(accuracy,feed_dict={x_:batch_x,y_:batch_y})
        print (i//(X.shape[0]//batch+1),res)


0 0.0625
1 0.09375
2 0.140625
3 0.234375
4 0.28125
5 0.34375
6 0.390625
7 0.484375
8 0.484375
9 0.53125
10 0.546875
11 0.59375
12 0.640625
13 0.703125
14 0.609375
15 0.671875
16 0.65625
17 0.71875
18 0.8125
19 0.859375
20 0.78125
21 0.84375
22 0.828125
23 0.859375
24 0.78125
25 0.828125
26 0.765625
27 0.765625
28 0.859375
29 0.890625
30 0.890625
31 0.875
32 0.90625
33 0.921875
34 0.953125
35 0.90625
36 0.84375
37 0.859375
38 0.90625
39 0.796875
40 0.875
41 0.859375
42 0.8125
43 0.90625
44 0.953125
45 0.953125
46 0.9375
47 0.984375
48 0.890625
49 0.953125
50 0.890625
51 0.9375
52 0.8125
53 0.875
54 0.921875
55 0.953125
56 0.90625
57 0.984375
58 0.9375
59 0.984375
60 0.953125
61 0.875
62 0.953125
63 0.921875
64 0.875
65 0.90625
66 0.90625
67 0.875
68 0.875
69 0.9375
70 0.921875
71 0.921875
72 0.984375
73 0.9375
74 0.96875
75 0.984375
76 0.9375
77 0.984375
78 0.9375
79 0.953125
80 0.984375
81 0.90625
82 0.953125
83 0.9375
84 0.96875
85 0.9375
86 0.953125
87 0.953125
88 0.984375
89 0.96875

In [60]:
sess.run(pred,feed_dict={x_:X[0:4]})[0]

array([  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   5.40643824e-32,
         1.82045735e-28,   0.00000000e+00,   0.00000000e+00,
         1.06261785e-34], dtype=float32)

NameError: name 'ProgressBar' is not defined

In [49]:
y[0]

matrix([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.layers.convolutional import Conv2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.imshow(X[1].reshape(8,8), cmap='Greys', interpolation='none')

In [ ]:
K.clear_session()
model = Sequential()

# Add model here
model.add(Conv2D(32,(3, 3), input_shape=X.shape[1:],activation="relu"))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(10,activation="softmax"))

# You can also try different optimizers below
model.compile(loss='categorical_crossentropy', 
              optimizer="adam", 
              metrics=['accuracy'])
print(model.summary())

In [ ]:
%%time

# Training
epochs = 25

history = model.fit(X, 
                    y, 
                    epochs=epochs, 
                    batch_size=128,
                    verbose=1)

In [ ]:
model.predict(X[0].reshape(1,8,8,1))